In [1]:
import re
import random
import numpy as np
import pandas as pd

In [193]:
data = "/Movie/MovieRecommender/resources/MovieDataset.xlsx"
df = pd.read_excel(data)

In [194]:
df = df.rename(columns={'Series or Movie': 'Type'})

In [195]:
df.head()

,Title,Genre,Tags,Languages,Type,Country Availability,Runtime,Director,Writer,Actors,...,Boxoffice,Release Date,Production House,Netflix Link,IMDb Link,Summary,IMDb Votes,Image,Poster,TMDb Trailer
0,Lets Fight Ghost,"Crime, Drama, Fantasy, Horror, Romance","Comedy Programmes,Romantic TV Comedies,Horror ...","Swedish, Spanish",Series,Thailand,< 30 minutes,Tomas Alfredson,John Ajvide Lindqvist,"Lina Leandersson, Kåre Hedebrant, Per Ragnar, ...",...,2122065.0,2008-12-12,"Canal+, Sandrew Metronome",https://www.netflix.com/watch/81415947,https://www.imdb.com/title/tt1139797,A med student with a supernatural gift tries t...,205926.0,https://occ-0-4708-64.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BOWM4NT...,https://www.youtube.com/watch?v=LqB6XJix-dM
1,HOW TO BUILD A GIRL,Comedy,"Dramas,Comedies,Films Based on Books,British",English,Movie,Canada,1-2 hour,Coky Giedroyc,Caitlin Moran,"Cleo, Paddy Considine, Beanie Feldstein, Dónal...",...,70632.0,2020-05-08,"Film 4, Monumental Pictures, Lionsgate",https://www.netflix.com/watch/81041267,https://www.imdb.com/title/tt4193072,"When nerdy Johanna moves to London, things get...",2838.0,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BZGUyN2...,https://www.youtube.com/watch?v=eIbcxPy4okQ
2,The Con-Heartist,"Comedy, Romance","Romantic Comedies,Comedies,Romantic Films,Thai...",Thai,Movie,Thailand,> 2 hrs,Mez Tharatorn,"Pattaranad Bhiboonsawade, Mez Tharatorn, Thods...","Kathaleeya McIntosh, Nadech Kugimiya, Pimchano...",...,NaN,2020-12-03,NaN,https://www.netflix.com/watch/81306155,https://www.imdb.com/title/tt13393728,After her ex-boyfriend cons her out of a large...,131.0,https://occ-0-2188-64.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BODAzOG...,https://www.youtube.com/watch?v=md3CmFLGK6Y
3,Gleboka woda,Drama,"TV Dramas,Polish TV Shows,Social Issue TV Dramas",Polish,Series,Poland,< 30 minutes,NaN,NaN,"Katarzyna Maciag, Piotr Nowak, Marcin Dorocins...",...,NaN,2011-06-14,NaN,https://www.netflix.com/watch/81307527,https://www.imdb.com/title/tt2300049,A group of social welfare workers led by their...,47.0,https://occ-0-2508-2706.1.nflxso.net/dnm/api/v...,https://m.media-amazon.com/images/M/MV5BMTc0Nz...,https://www.youtube.com/watch?v=5kyF2vy63r0
4,Only a Mother,Drama,"Social Issue Dramas,Dramas,Movies Based on Boo...",Swedish,Movie,"Lithuania,Poland,France,Italy,Spain,Greece,Bel...",1-2 hour,Alf Sjöberg,Ivar Lo-Johansson,"Hugo Björne, Eva Dahlbeck, Ulf Palme, Ragnar F...",...,NaN,1949-10-31,NaN,https://www.netflix.com/watch/81382068,https://www.imdb.com/title/tt0041155,An unhappily married farm worker struggling to...,88.0,https://occ-0-2851-41.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BMjVmMz...,https://www.youtube.com/watch?v=H0itWKFwMpQ


In [196]:
# count the null values in each column
null_counts = df.isnull().sum()
total_rows = len(df)
null_percentages = (null_counts / total_rows) * 100
print("Null Value Counts and Percentages:")
for column, count in null_counts.items():
    percentage = null_percentages[column]
    print(f"{column}: {count} ({percentage:.2f}%)")

Null Value Counts and Percentages:
Title: 0 (0.00%)
Genre: 25 (0.27%)
Tags: 36 (0.38%)
Languages: 159 (1.69%)
Type: 0 (0.00%)
Country Availability: 11 (0.12%)
Runtime: 1 (0.01%)
Director: 2305 (24.46%)
Writer: 1810 (19.20%)
Actors: 111 (1.18%)
IMDb Score: 8 (0.08%)
Rotten Tomatoes Score: 3980 (42.23%)
Metacritic Score: 5343 (56.69%)
Awards Received: 4199 (44.55%)
Awards Nominated For: 3049 (32.35%)
Boxoffice: 5671 (60.17%)
Release Date: 208 (2.21%)
Production House: 5032 (53.39%)
Netflix Link: 0 (0.00%)
IMDb Link: 324 (3.44%)
Summary: 5 (0.05%)
IMDb Votes: 10 (0.11%)
Image: 0 (0.00%)
Poster: 938 (9.95%)
TMDb Trailer: 0 (0.00%)


In [197]:
# null threshold
null_column_threshold = len(df) * 0.5
# null values in each column
null_counts = df.isnull().sum()
# columns with at least 50% null values
columns_to_delete = null_counts[null_counts >= null_column_threshold].index
df = df.drop(columns=columns_to_delete)

In [198]:
df.columns

Index(['Title', 'Genre', 'Tags', 'Languages', 'Type', 'Country Availability',
       'Runtime', 'Director', 'Writer', 'Actors', 'IMDb Score',
       'Rotten Tomatoes Score', 'Awards Received', 'Awards Nominated For',
       'Release Date', 'Netflix Link', 'IMDb Link', 'Summary', 'IMDb Votes',
       'Image', 'Poster', 'TMDb Trailer'],
      dtype='object')

In [199]:
# threshold for null values (50% of the columns)
null_threshold = len(df.columns) / 2
# count the null values in each row
null_counts_per_row = df.isnull().sum(axis=1)

for index, count in null_counts_per_row.iteritems():
    if count >= null_threshold:
        total_columns = len(df.columns)
        print(f"Row {index}: {count} null values out of {total_columns} column numbers")

Row 5208: 11 null values out of 22 column numbers


In [200]:
rows_to_delete = null_counts_per_row >= null_threshold
# delete the rows that meet the null threshold
df = df[~rows_to_delete]
df = df.reset_index(drop=True)

In [201]:
max_genres = df['Genre'].str.split(',', expand=True).apply(lambda x: x.notna().sum(), axis=1).max()
genre_columns = [f'Genre{i+1}' for i in range(max_genres)]
split_genres = df['Genre'].str.split(',', expand=True)
split_genres.columns = genre_columns

selected_genre_columns = []
for _, row in split_genres.iterrows():
    non_null_genres = row.dropna().values
    genre_count = len(non_null_genres)
    if genre_count > 0:
        if genre_count >= 3:
            selected_genres = np.random.choice(non_null_genres, size=3, replace=False)
        else:
            selected_genres = np.random.choice(non_null_genres, size=3, replace=True)
        selected_genre_columns.append(selected_genres)
    else:
        selected_genre_columns.append([None] * 3)

selected_genre_columns = np.array(selected_genre_columns)
df = pd.concat([df, pd.DataFrame(selected_genre_columns, columns=['Genre1', 'Genre2', 'Genre3'])], axis=1)

In [202]:
df = df.drop(columns=['Genre'])

In [203]:
df.columns

Index(['Title', 'Tags', 'Languages', 'Type', 'Country Availability', 'Runtime',
       'Director', 'Writer', 'Actors', 'IMDb Score', 'Rotten Tomatoes Score',
       'Awards Received', 'Awards Nominated For', 'Release Date',
       'Netflix Link', 'IMDb Link', 'Summary', 'IMDb Votes', 'Image', 'Poster',
       'TMDb Trailer', 'Genre1', 'Genre2', 'Genre3'],
      dtype='object')

In [204]:
genre_columns = ['Genre1', 'Genre2', 'Genre3']
unique_genres = pd.unique(df[genre_columns].apply(lambda x: x.str.strip()).values.ravel())
unique_genres

array(['Romance', 'Drama', 'Horror', 'Comedy', 'Crime', 'Fantasy',
       'Short', 'Thriller', 'Action', 'Sci-Fi', 'Mystery', 'Music',
       'Animation', 'Family', 'Adventure', 'War', 'History',
       'Documentary', 'Biography', 'Film-Noir', 'Sport', 'Game-Show',
       'Western', None, 'Reality-TV', 'Musical', 'News', 'Talk-Show',
       'Adult'], dtype=object)

In [205]:
unique_genres = [genre for genre in unique_genres if genre is not None]
genre_df = pd.DataFrame({'Genre': unique_genres})
genre_df['GenreID'] = range(1, len(genre_df) + 1)
genre_df['GenreID'] = genre_df['GenreID'].astype(int)
genre_df

,Genre,GenreID
0,Romance,1
1,Drama,2
2,Horror,3
3,Comedy,4
4,Crime,5
5,Fantasy,6
6,Short,7
7,Thriller,8
8,Action,9
9,Sci-Fi,10


In [206]:
genre_dict = {genre: genre_id for genre, genre_id in zip(genre_df['Genre'], genre_df['GenreID'])}
genre_dict[None] = None
genre_dict

{'Romance': 1,
 'Drama': 2,
 'Horror': 3,
 'Comedy': 4,
 'Crime': 5,
 'Fantasy': 6,
 'Short': 7,
 'Thriller': 8,
 'Action': 9,
 'Sci-Fi': 10,
 'Mystery': 11,
 'Music': 12,
 'Animation': 13,
 'Family': 14,
 'Adventure': 15,
 'War': 16,
 'History': 17,
 'Documentary': 18,
 'Biography': 19,
 'Film-Noir': 20,
 'Sport': 21,
 'Game-Show': 22,
 'Western': 23,
 'Reality-TV': 24,
 'Musical': 25,
 'News': 26,
 'Talk-Show': 27,
 'Adult': 28,
 None: None}

In [207]:
df.iloc[9392]

Title                                Spy Kids 2: The Island of Lost Dreams
Tags                     Family Features,Family Comedies,Family Adventu...
Languages                                                          English
Type                                                                 Movie
Country Availability                                                Canada
Runtime                                                           1-2 hour
Director                                                  Robert Rodriguez
Writer                                                    Robert Rodriguez
Actors                   Daryl Sabara, Carla Gugino, Alexa PenaVega, An...
IMDb Score                                                             5.2
Rotten Tomatoes Score                                                 75.0
Awards Received                                                        3.0
Awards Nominated For                                                   3.0
Release Date             

In [208]:
value_columns = []
split_values = pd.DataFrame()

# iterate over each column and split values
for column in ['Tags', 'Languages', 'Director', 'Actors']:
    # max number of values in the column
    max_values = df[column].str.split(',', expand=False).apply(lambda x: len(x) if isinstance(x, list) else 1).max()
    if column == 'Director':
        # limit the maximum number of directors to 2
        max_values = 2
    
    value_columns.extend([f'{column}_{i+1}' for i in range(max_values)])
    split_column = df[column].str.split(',', expand=True)
    
    # limit each row to a maximum of 2 directors
    split_column = split_column.apply(lambda row: row.dropna().tolist()[:2], axis=1)
    split_column = split_column.apply(pd.Series)
    
    # add None values if there are less than 2 directors
    if max_values > 2:
        missing_cols = [f'{column}_{i+3}' for i in range(max_values-2)]
        split_column = pd.concat([split_column, pd.DataFrame(columns=missing_cols)], axis=1)
    
    split_column.columns = [f'{column}_{i+1}' for i in range(max_values)]
    
    # remove leading and trailing spaces from split values
    split_column = split_column.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    split_values = pd.concat([split_values, split_column], axis=1)

df = pd.concat([df, split_values], axis=1)

C:\Users\danar\AppData\Local\Temp\ipykernel_6180\798575545.py:17: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  split_column = split_column.apply(pd.Series)
C:\Users\danar\AppData\Local\Temp\ipykernel_6180\798575545.py:17: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  split_column = split_column.apply(pd.Series)
C:\Users\danar\AppData\Local\Temp\ipykernel_6180\798575545.py:17: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  split_column = split_column.apply(pd.Series)
C:\Users\danar\AppData\Local\Temp\ipykernel_6180\798575545.py:17: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Spe

In [209]:
df.columns

Index(['Title', 'Tags', 'Languages', 'Type', 'Country Availability', 'Runtime',
       'Director', 'Writer', 'Actors', 'IMDb Score', 'Rotten Tomatoes Score',
       'Awards Received', 'Awards Nominated For', 'Release Date',
       'Netflix Link', 'IMDb Link', 'Summary', 'IMDb Votes', 'Image', 'Poster',
       'TMDb Trailer', 'Genre1', 'Genre2', 'Genre3', 'Tags_1', 'Tags_2',
       'Tags_3', 'Tags_4', 'Tags_5', 'Tags_6', 'Tags_7', 'Tags_8', 'Tags_9',
       'Tags_10', 'Tags_11', 'Tags_12', 'Tags_13', 'Tags_14', 'Tags_15',
       'Tags_16', 'Tags_17', 'Tags_18', 'Tags_19', 'Tags_20', 'Tags_21',
       'Tags_22', 'Tags_23', 'Tags_24', 'Tags_25', 'Tags_26', 'Tags_27',
       'Tags_28', 'Tags_29', 'Tags_30', 'Tags_31', 'Tags_32', 'Tags_33',
       'Languages_1', 'Languages_2', 'Languages_3', 'Languages_4',
       'Languages_5', 'Languages_6', 'Languages_7', 'Languages_8',
       'Languages_9', 'Languages_10', 'Languages_11', 'Languages_12',
       'Languages_13', 'Languages_14', 'Director_1

In [210]:
columns_to_process = ['Languages', 'Country Availability', 'Director', 'Actors']
language_df = pd.DataFrame(columns=['Language', 'LanguageID'])
country_df = pd.DataFrame(columns=['Country', 'CountryID'])
director_df = pd.DataFrame(columns=['Director', 'DirectorID'])
actor_df = pd.DataFrame(columns=['Actor', 'ActorID'])

for column in columns_to_process:
    # remove null and empty values
    values = df[column].dropna().str.strip()
    # unique values
    unique_values = values.str.split(',', expand=True).stack().str.strip().unique()
    # dataFrame with values and IDs
    df_column = pd.DataFrame({column: unique_values})
    df_column['ID'] = range(1, len(df_column) + 1)
    # store DataFrame in corresponding variable
    if column == 'Languages':
        language_df = df_column.rename(columns={column: 'Language', 'ID': 'LanguageID'})
    elif column == 'Country Availability':
        country_df = df_column.rename(columns={column: 'Country', 'ID': 'CountryID'})
    elif column == 'Director':
        director_df = df_column.rename(columns={column: 'Director', 'ID': 'DirectorID'})
    elif column == 'Actors':
        actor_df = df_column.rename(columns={column: 'Actor', 'ID': 'ActorID'})

In [211]:
language_df

,Language,LanguageID
0,Swedish,1
1,Spanish,2
2,English,3
3,Thai,4
4,Polish,5
...,...,...
179,Norse,180
180,Old,181
181,Kinyarwanda,182
182,Quenya,183


In [212]:
language_df['Language'] = language_df['Language'].astype(str).str.strip()
language_df['LanguageID'] = language_df['LanguageID'].astype(str).str.strip()

country_df['Country'] = country_df['Country'].astype(str).str.strip()
country_df['CountryID'] = country_df['CountryID'].astype(str).str.strip()

director_df['Director'] = director_df['Director'].astype(str).str.strip()
director_df['DirectorID'] = director_df['DirectorID'].astype(str).str.strip()

actor_df['Actor'] = actor_df['Actor'].astype(str).str.strip()
actor_df['ActorID'] = actor_df['ActorID'].astype(str).str.strip()

In [213]:
language_dict = language_df.set_index('Language')['LanguageID'].to_dict()
country_dict = country_df.set_index('Country')['CountryID'].to_dict()
director_dict = director_df.set_index('Director')['DirectorID'].to_dict()
actor_dict = actor_df.set_index('Actor')['ActorID'].to_dict()

In [214]:
language_dict

{'Swedish': '1',
 'Spanish': '2',
 'English': '3',
 'Thai': '4',
 'Polish': '5',
 'German': '6',
 'Norwegian': '7',
 'Scanian': '8',
 'Sanskrit': '9',
 'Japanese': '10',
 'Cantonese': '11',
 'Mandarin': '12',
 'French': '13',
 'Italian': '14',
 'Breton': '15',
 'Korean': '16',
 'Tibetan': '17',
 'Russian': '18',
 'Danish': '19',
 'Romanian': '20',
 'None': '21',
 'Hindi': '22',
 'Hungarian': '23',
 'Swiss German': '24',
 'Latin': '25',
 'Vietnamese': '26',
 'Luxembourgish': '27',
 'Indonesian': '28',
 'Portuguese': '29',
 'Min Nan': '30',
 'Arabic': '31',
 'Turkish': '32',
 'Filipino': '33',
 'Romany': '34',
 'American Sign Language': '35',
 'Quechua': '36',
 'Dutch': '37',
 'Khmer': '38',
 'Hokkien': '39',
 'Tagalog': '40',
 'Yiddish': '41',
 'Hebrew': '42',
 'Chinese': '43',
 'Gallegan': '44',
 'Finnish': '45',
 'Malay': '46',
 'Sign Languages': '47',
 'Georgian': '48',
 'Greek': '49',
 'Tamil': '50',
 'Persian': '51',
 'Inuktitut': '52',
 'Mongolian': '53',
 'Serbian': '54',
 'Ukrai

In [215]:
unique_types = df['Type'].unique()
types_df = pd.DataFrame({'type': unique_types})
types_df['typeid'] = types_df.index + 1

In [216]:
# convert "Release Date" column to datetime type
df['Release Date'] = pd.to_datetime(df['Release Date'])
# extract year 
df['Year'] = df['Release Date'].dt.year
df = df.drop(columns=['Release Date'])
df['Year'] = df['Year'].fillna(0).astype(int)

In [217]:
columns_to_drop = ['Languages', 'Country Availability', 'Director', 'Writer', 'Actors']
df = df.drop(columns=columns_to_drop)

In [218]:
df = df.replace(np.nan, None)

In [219]:
unique_values = df['Runtime'].unique()
unique_values

array(['< 30 minutes', '1-2 hour', '> 2 hrs', '30-60 mins', None],
      dtype=object)

In [220]:
df['Runtime'] = df['Runtime'].str.replace('[<>]', '', regex=True)

In [221]:
# calculate statistical information
rotten_score_column = df['Rotten Tomatoes Score']
rotten_score_count = rotten_score_column.count()
minimum_rotten_score = rotten_score_column.min()
maximum_rotten_score = rotten_score_column.max()
rotten_score_mean = rotten_score_column.mean()
print("Count:", rotten_score_count)
print("Mean:", rotten_score_mean)
print("Minimum:", minimum_rotten_score)
print("Maximum:", maximum_rotten_score)

Count: 5444
Mean: 64.69360764144012
Minimum: 0.0
Maximum: 100.0


In [222]:
IMDb_score_column = df['IMDb Score']
IMDb_score_count = IMDb_score_column.count()
minimum_IMDb_score = IMDb_score_column.min()
maximum_IMDb_score = IMDb_score_column.max()
IMDb_score_mean = IMDb_score_column.mean()
print("Count:", IMDb_score_count)
print("Mean:", IMDb_score_mean)
print("Minimum:", minimum_IMDb_score)
print("Maximum:", maximum_IMDb_score)

Count: 9417
Mean: 6.95551661888072
Minimum: 1.6
Maximum: 9.7


In [223]:
awards_recieved_column = df['Awards Received']
awards_recieved_count = awards_recieved_column.count()
minimum_awards_recieved = awards_recieved_column.min()
maximum_awards_recieved = awards_recieved_column.max()
awards_recieved_mean = awards_recieved_column.mean()
print("Count:", awards_recieved_count)
print("Mean:", awards_recieved_mean)
print("Minimum:", minimum_awards_recieved)
print("Maximum:", maximum_awards_recieved)

Count: 5226
Mean: 9.735935706084959
Minimum: 1.0
Maximum: 300.0


In [224]:
df['MovieID'] = range(1, len(df) + 1)

In [225]:
df.head()

,Title,Tags,Type,Runtime,IMDb Score,Rotten Tomatoes Score,Awards Received,Awards Nominated For,Netflix Link,IMDb Link,...,Languages_13,Languages_14,Director_1,Director_2,Actors_1,Actors_2,Actors_3,Actors_4,Year,MovieID
0,Lets Fight Ghost,"Comedy Programmes,Romantic TV Comedies,Horror ...",Series,30 minutes,7.9,98.0,74.0,57.0,https://www.netflix.com/watch/81415947,https://www.imdb.com/title/tt1139797,...,None,None,Tomas Alfredson,None,Lina Leandersson,Kåre Hedebrant,None,None,2008,1
1,HOW TO BUILD A GIRL,"Dramas,Comedies,Films Based on Books,British",Movie,1-2 hour,5.8,79.0,1.0,None,https://www.netflix.com/watch/81041267,https://www.imdb.com/title/tt4193072,...,None,None,Coky Giedroyc,None,Cleo,Paddy Considine,None,None,2020,2
2,The Con-Heartist,"Romantic Comedies,Comedies,Romantic Films,Thai...",Movie,2 hrs,7.4,None,None,None,https://www.netflix.com/watch/81306155,https://www.imdb.com/title/tt13393728,...,None,None,Mez Tharatorn,None,Kathaleeya McIntosh,Nadech Kugimiya,None,None,2020,3
3,Gleboka woda,"TV Dramas,Polish TV Shows,Social Issue TV Dramas",Series,30 minutes,7.5,None,2.0,4.0,https://www.netflix.com/watch/81307527,https://www.imdb.com/title/tt2300049,...,None,None,None,None,Katarzyna Maciag,Piotr Nowak,None,None,2011,4
4,Only a Mother,"Social Issue Dramas,Dramas,Movies Based on Boo...",Movie,1-2 hour,6.7,None,2.0,1.0,https://www.netflix.com/watch/81382068,https://www.imdb.com/title/tt0041155,...,None,None,Alf Sjöberg,None,Hugo Björne,Eva Dahlbeck,None,None,1949,5


In [226]:
df = df.reset_index(drop=True)

In [227]:
imdb_threshold = 8
rotten_threshold = 80
awards_threshold = awards_recieved_mean
# filter popular movies based on the criteria
popular_movies = df[(df['IMDb Score'] >= imdb_threshold) &
                    (df['Rotten Tomatoes Score'] >= rotten_threshold) &
                    (df['Awards Received'] + df['Awards Nominated For'] >= awards_threshold)]
popular_movies_df = popular_movies['MovieID']

In [228]:
popular_movies_df

11        12
16        17
46        47
188      189
262      263
        ... 
9339    9340
9344    9345
9354    9355
9365    9366
9384    9385
Name: MovieID, Length: 215, dtype: int64

In [229]:
current_year = 2023
trending_years = [current_year - 1, current_year - 2, current_year - 3]
# filter trending movies based on the release year
trending_movies = df[df['Year'].isin(trending_years)]
trending_movies_dataset = trending_movies['MovieID']

In [230]:
trending_movies_dataset

1          2
2          3
15        16
25        26
30        31
        ... 
6027    6028
6209    6210
7318    7319
7343    7344
7412    7413
Name: MovieID, Length: 529, dtype: int64

In [231]:
df.head()

,Title,Tags,Type,Runtime,IMDb Score,Rotten Tomatoes Score,Awards Received,Awards Nominated For,Netflix Link,IMDb Link,...,Languages_13,Languages_14,Director_1,Director_2,Actors_1,Actors_2,Actors_3,Actors_4,Year,MovieID
0,Lets Fight Ghost,"Comedy Programmes,Romantic TV Comedies,Horror ...",Series,30 minutes,7.9,98.0,74.0,57.0,https://www.netflix.com/watch/81415947,https://www.imdb.com/title/tt1139797,...,None,None,Tomas Alfredson,None,Lina Leandersson,Kåre Hedebrant,None,None,2008,1
1,HOW TO BUILD A GIRL,"Dramas,Comedies,Films Based on Books,British",Movie,1-2 hour,5.8,79.0,1.0,None,https://www.netflix.com/watch/81041267,https://www.imdb.com/title/tt4193072,...,None,None,Coky Giedroyc,None,Cleo,Paddy Considine,None,None,2020,2
2,The Con-Heartist,"Romantic Comedies,Comedies,Romantic Films,Thai...",Movie,2 hrs,7.4,None,None,None,https://www.netflix.com/watch/81306155,https://www.imdb.com/title/tt13393728,...,None,None,Mez Tharatorn,None,Kathaleeya McIntosh,Nadech Kugimiya,None,None,2020,3
3,Gleboka woda,"TV Dramas,Polish TV Shows,Social Issue TV Dramas",Series,30 minutes,7.5,None,2.0,4.0,https://www.netflix.com/watch/81307527,https://www.imdb.com/title/tt2300049,...,None,None,None,None,Katarzyna Maciag,Piotr Nowak,None,None,2011,4
4,Only a Mother,"Social Issue Dramas,Dramas,Movies Based on Boo...",Movie,1-2 hour,6.7,None,2.0,1.0,https://www.netflix.com/watch/81382068,https://www.imdb.com/title/tt0041155,...,None,None,Alf Sjöberg,None,Hugo Björne,Eva Dahlbeck,None,None,1949,5


In [232]:
df[genre_columns] 

,Genre1,Genre2,Genre3
0,Romance,Drama,Horror
1,Comedy,Comedy,Comedy
2,Comedy,Romance,Comedy
3,Drama,Drama,Drama
4,Drama,Drama,Drama
...,...,...,...
9419,Fantasy,Comedy,Romance
9420,Documentary,Documentary,Documentary
9421,Music,Documentary,Music
9422,Comedy,Comedy,Comedy


In [233]:
columns_to_keep = ['Languages_1', 'Languages_2']
# columns to drop
columns_to_drop = ['Languages_{}'.format(i) for i in range(3, 15)]
df = df.drop(columns_to_drop, axis=1)
df = df.rename(columns={'Languages_1': 'language1', 'Languages_2': 'language2'})

In [234]:
columns_to_drop = ['Rotten Tomatoes Score', 'Awards Received', 'Awards Nominated For']
tags_columns_to_drop = [f'Tags_{i}' for i in range(7, 34)]
columns_to_drop += tags_columns_to_drop
df = df.drop(columns_to_drop, axis=1)

In [235]:
df.columns

Index(['Title', 'Tags', 'Type', 'Runtime', 'IMDb Score', 'Netflix Link',
       'IMDb Link', 'Summary', 'IMDb Votes', 'Image', 'Poster', 'TMDb Trailer',
       'Genre1', 'Genre2', 'Genre3', 'Tags_1', 'Tags_2', 'Tags_3', 'Tags_4',
       'Tags_5', 'Tags_6', 'language1', 'language2', 'Director_1',
       'Director_2', 'Actors_1', 'Actors_2', 'Actors_3', 'Actors_4', 'Year',
       'MovieID'],
      dtype='object')

In [236]:
trending_movies_dataset 

1          2
2          3
15        16
25        26
30        31
        ... 
6027    6028
6209    6210
7318    7319
7343    7344
7412    7413
Name: MovieID, Length: 529, dtype: int64

In [237]:
trending_movies_df = trending_movies_dataset.to_frame().rename(columns={0: 'MovieID'})

In [238]:
popular_movies_df = popular_movies_df.to_frame().rename(columns={0: 'MovieID'})

In [239]:
genre_dict

{'Romance': 1,
 'Drama': 2,
 'Horror': 3,
 'Comedy': 4,
 'Crime': 5,
 'Fantasy': 6,
 'Short': 7,
 'Thriller': 8,
 'Action': 9,
 'Sci-Fi': 10,
 'Mystery': 11,
 'Music': 12,
 'Animation': 13,
 'Family': 14,
 'Adventure': 15,
 'War': 16,
 'History': 17,
 'Documentary': 18,
 'Biography': 19,
 'Film-Noir': 20,
 'Sport': 21,
 'Game-Show': 22,
 'Western': 23,
 'Reality-TV': 24,
 'Musical': 25,
 'News': 26,
 'Talk-Show': 27,
 'Adult': 28,
 None: None}

In [240]:
df['Genre1']

0           Romance
1            Comedy
2            Comedy
3             Drama
4             Drama
           ...     
9419        Fantasy
9420    Documentary
9421          Music
9422         Comedy
9423         Family
Name: Genre1, Length: 9424, dtype: object

In [241]:
# Remove spaces from Genre1, Genre2, and Genre3 columns
df['Genre1'] = df['Genre1'].str.replace(' ', '')
df['Genre2'] = df['Genre2'].str.replace(' ', '')
df['Genre3'] = df['Genre3'].str.replace(' ', '')

# Replace Genre1, Genre2, and Genre3 values with dictionary values
df['Genre1'] = df['Genre1'].replace(genre_dict)
df['Genre2'] = df['Genre2'].replace(genre_dict)
df['Genre3'] = df['Genre3'].replace(genre_dict)

In [242]:
df['Genre1']

0        1.0
1        4.0
2        4.0
3        2.0
4        2.0
        ... 
9419     6.0
9420    18.0
9421    12.0
9422     4.0
9423    14.0
Name: Genre1, Length: 9424, dtype: object

In [243]:
df['Genre1'] = df['Genre1'].fillna(-1).astype(int)
df['Genre2'] = df['Genre2'].fillna(-1).astype(int)
df['Genre3'] = df['Genre3'].fillna(-1).astype(int)

In [244]:
df['Genre1'] = df['Genre1'].fillna(-1).astype(int).replace(-1, None)
df['Genre2'] = df['Genre2'].fillna(-1).astype(int).replace(-1, None)
df['Genre3'] = df['Genre3'].fillna(-1).astype(int).replace(-1, None)

In [249]:
# Get the unique genre IDs from the movies DataFrame
movie_genre_ids = set(df['Genre1'].unique()).union(set(df['Genre2'].unique())).union(set(df['Genre3'].unique()))

# Get the unique genre IDs from the genres DataFrame
genre_ids = set(genre_df['GenreID'].unique())

# Find invalid genre IDs in the movies DataFrame
invalid_genre_ids = movie_genre_ids - genre_ids

# Find missing genre IDs in the movies DataFrame
missing_genre_ids = genre_ids - movie_genre_ids

print("Invalid genre IDs in movies.csv:", invalid_genre_ids)
print("Missing genre IDs in movies.csv:", missing_genre_ids)

Invalid genre IDs in movies.csv: {None}
Missing genre IDs in movies.csv: set()


In [247]:
# Replace 'None' with NaN in Genre1, Genre2, and Genre3 columns
df['Genre1'] = df['Genre1'].replace('None', np.nan)
df['Genre2'] = df['Genre2'].replace('None', np.nan)
df['Genre3'] = df['Genre3'].replace('None', np.nan)

In [248]:
df.to_csv('movies.csv', index=False)
popular_movies_df.to_csv('popular_movies.csv', index=False)
trending_movies_df.to_csv('trending_movies.csv', index=False)
genre_df.to_csv('genres.csv', index=False)

In [2]:
import pandas as pd
movies_  = pd.read_csv('movies.csv')
movies_.head()

,Title,Type,Runtime,IMDb_Score,Netflix_Link,IMDb_Link,Summary,IMDb_Votes,Image,Poster,...,language1,language2,Director_1,Director_2,Actors_1,Actors_2,Actors_3,Actors_4,Year,MovieID
0,Lets Fight Ghost,Series,30 minutes,7.9,https://www.netflix.com/watch/81415947,https://www.imdb.com/title/tt1139797,A med student with a supernatural gift tries t...,205926.0,https://occ-0-4708-64.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BOWM4NT...,...,Swedish,Spanish,Tomas Alfredson,NaN,Lina Leandersson,Kåre Hedebrant,NaN,NaN,2008,1
1,HOW TO BUILD A GIRL,Movie,1-2 hour,5.8,https://www.netflix.com/watch/81041267,https://www.imdb.com/title/tt4193072,"When nerdy Johanna moves to London, things get...",2838.0,https://occ-0-1081-999.1.nflxso.net/dnm/api/v6...,https://m.media-amazon.com/images/M/MV5BZGUyN2...,...,English,NaN,Coky Giedroyc,NaN,Cleo,Paddy Considine,NaN,NaN,2020,2
2,The Con-Heartist,Movie,2 hrs,7.4,https://www.netflix.com/watch/81306155,https://www.imdb.com/title/tt13393728,After her ex-boyfriend cons her out of a large...,131.0,https://occ-0-2188-64.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BODAzOG...,...,Thai,NaN,Mez Tharatorn,NaN,Kathaleeya McIntosh,Nadech Kugimiya,NaN,NaN,2020,3
3,Gleboka woda,Series,30 minutes,7.5,https://www.netflix.com/watch/81307527,https://www.imdb.com/title/tt2300049,A group of social welfare workers led by their...,47.0,https://occ-0-2508-2706.1.nflxso.net/dnm/api/v...,https://m.media-amazon.com/images/M/MV5BMTc0Nz...,...,Polish,NaN,NaN,NaN,Katarzyna Maciag,Piotr Nowak,NaN,NaN,2011,4
4,Only a Mother,Movie,1-2 hour,6.7,https://www.netflix.com/watch/81382068,https://www.imdb.com/title/tt0041155,An unhappily married farm worker struggling to...,88.0,https://occ-0-2851-41.1.nflxso.net/dnm/api/v6/...,https://m.media-amazon.com/images/M/MV5BMjVmMz...,...,Swedish,NaN,Alf Sjöberg,NaN,Hugo Björne,Eva Dahlbeck,NaN,NaN,1949,5


In [4]:
movies_.columns = movies_.columns.str.replace(' ', '_')

In [5]:
movies_.columns

Index(['Title', 'Type', 'Runtime', 'IMDb_Score', 'Netflix_Link', 'IMDb_Link',
       'Summary', 'IMDb_Votes', 'Image', 'Poster', 'TMDb_Trailer', 'Genre1',
       'Genre2', 'Genre3', 'Tags_1', 'Tags_2', 'Tags_3', 'Tags_4', 'Tags_5',
       'Tags_6', 'language1', 'language2', 'Director_1', 'Director_2',
       'Actors_1', 'Actors_2', 'Actors_3', 'Actors_4', 'Year', 'MovieID'],
      dtype='object')

In [12]:
print(movies_['Genre1'].unique())
print(movies_['Genre2'].unique())
print(movies_['Genre3'].unique())

[1 4 2 5 6 11 12 10 14 15 16 17 18 19 9 3 20 8 13 21 None 7 23 24 26 25 22
 27 28]
[2 4 1 7 5 9 12 11 16 18 19 8 15 3 6 13 21 22 23 14 10 None 24 17 27 25 28
 26]
[3 4 2 6 8 10 12 13 1 16 19 18 5 15 17 9 21 14 11 23 7 None 22 24 25 26 27
 28]


In [10]:
movies_['Genre1'] = movies_['Genre1'].fillna(0).astype(int)
movies_['Genre2'] = movies_['Genre2'].fillna(0).astype(int)
movies_['Genre3'] = movies_['Genre3'].fillna(0).astype(int)

In [11]:
movies_['Genre1'] = movies_['Genre1'].replace(0, None)
movies_['Genre2'] = movies_['Genre2'].replace(0, None)
movies_['Genre3'] = movies_['Genre3'].replace(0, None)

In [14]:
movie_title = "Partners: The Movie IV"
matching_movies = movies_[movies_['Title'] == movie_title]
index = matching_movies.index[0]
row = matching_movies.iloc[0]
print("Index:", index)
print("Row:")
print(row)

Index: 169
Row:
Title                                      Partners: The Movie IV
Type                                                        Movie
Runtime                                                     2 hrs
IMDb_Score                                                    6.6
Netflix_Link               https://www.netflix.com/watch/81399193
IMDb_Link                    https://www.imdb.com/title/tt6857258
Summary         The leader of an international criminal organi...
IMDb_Votes                                                    5.0
Image           https://occ-0-3187-3188.1.nflxso.net/dnm/api/v...
Poster          https://m.media-amazon.com/images/M/MV5BNzc1Mj...
TMDb_Trailer          https://www.youtube.com/watch?v=abCAeEkJY_4
Genre1                                                       None
Genre2                                                       None
Genre3                                                       None
Tags_1                                            Thriller M

In [15]:
movies_.drop(index, inplace=True)
movies_.reset_index(drop=True, inplace=True)
movies_.drop('MovieID', axis=1, inplace=True)

In [17]:
movies_ = movies_.drop('IMDb_Votes', axis=1)

In [18]:
movies_.to_csv('movies.csv', index=False)

In [3]:
import pandas as pd
popular_ids = pd.read_csv('popular_movies.csv')
movies_ = pd.read_csv('movies.csv')